## First true test: can a simple R-D system for a self-limiting actication domain?

Define a function that gives the logic of the PDE system

In [ ]:
from pde import ScalarField, FieldCollection, PDEBase
import numpy as np

class TuringPDE(PDEBase):
    def __init__(self,
                 da=1.0,
                 lambda_=10.0,
                 k_a=1e-4,
                 k_r=0.5e-4,
                 n=2, m=2, p=2,
                 bc="auto_periodic_neumann",
                 n_A_spots=5,
                 r_init="constant",
                 r_value=0.0,
                 r_noise_range=(0.2, 0.6)):
        """
        Activator–repressor PDE system with saturating Hill-like activation and repression.

        Parameters
        ----------
        da : float
            Diffusion coefficient of the activator.
        lambda_ : float
            Diffusivity ratio between repressor and activator: D_r = lambda_ * D_a.
        k_a : float
            Linear degradation rate of the activator.
        k_r : float
            Linear degradation rate of the repressor.
        n, m, p : int
            Hill coefficients for activator self-activation (n),
            repression by R (m), and activation of R by A (p).
        bc : str
            Boundary condition type. "periodic" or "auto_periodic_neumann" are typical.
        n_A_spots : int
            Number of Gaussian spots to initialize in activator field.
        r_init : str
            Repressor initialization method: "constant" or "random".
        r_value : float
            Initial value for repressor if `r_init = "constant"`.
        r_noise_range : tuple
            Range of uniform noise for repressor if `r_init = "random"`.
        """
        super().__init__()
        self.da = da
        self.dr = lambda_ * da
        self.k_a = k_a
        self.k_r = k_r
        self.n = n
        self.m = m
        self.p = p
        self.bc = bc
        self.n_A_spots = n_A_spots
        self.r_init = r_init
        self.r_value = r_value
        self.r_noise_range = r_noise_range

    def evolution_rate(self, state, t=0):
        A, R = state

        f_A = 1 * (A**self.n) / (1**self.n + A**self.n) * (1 / (1**self.m + R**self.m)) - self.k_a * A
        f_R = (A**self.p) / (1**self.p + A**self.p) - self.k_r * R

        dA_dt = self.da * A.laplace(self.bc) + f_A
        dR_dt = self.dr * R.laplace(self.bc) + f_R

        return FieldCollection([dA_dt, dR_dt])

    def get_state(self, grid):
        A = self.make_spot_field(grid, n_spots=self.n_A_spots,
                                 radius=25, amplitude_range=(2, 2.5)).copy(label="Activator")
        R = self.initialize_repressor(grid).copy(label="Repressor")
        return FieldCollection([A, R])

    def make_spot_field(self, grid, n_spots=3, radius=10, amplitude_range=(1, 1.1), seed=None):
        if seed is not None:
            np.random.seed(seed)

        field = ScalarField(grid, data=0.0)

        Nx, Ny = grid.shape
        (x_min, x_max), (y_min, y_max) = grid.axes_bounds
        x = np.linspace(x_min, x_max, Nx)
        y = np.linspace(y_min, y_max, Ny)
        X, Y = np.meshgrid(x, y, indexing="ij")

        for _ in range(n_spots):
            cx, cy = np.random.uniform(x_min, x_max), np.random.uniform(y_min, y_max)
            amplitude = np.random.uniform(*amplitude_range)
            dist2 = (X - cx)**2 + (Y - cy)**2
            field.data += amplitude * np.exp(-dist2 / (2 * radius**2))

        return field

    def initialize_repressor(self, grid):
        if self.r_init == "constant":
            return ScalarField(grid, data=self.r_value)
        elif self.r_init == "random":
            noise = np.random.uniform(*self.r_noise_range, size=grid.shape)
            return ScalarField(grid, data=noise)
        else:
            raise ValueError(f"Unrecognized r_init method: {self.r_init}")


In [ ]:
from pde import CartesianGrid, PlotTracker#, SphericalGrid
# from turing_model import TuringPDE  # adjust if class is defined inline
import matplotlib.pyplot as plt

# Set up spatial domain
L0 = 3000  # physical domain size (μm)
res = 25
N = L0 // res   # grid resolution
L = N*res
grid = CartesianGrid([[0, L], [0, L]], shape=(N, N), periodic=True)
# grid = SphericalSymGrid(radius=R, shape=N)

# Initialize PDE system
model = TuringPDE(
    da=1,
    lambda_=20.0,
    n=5,
    m=5,
    p=5,
    n_A_spots=1,
    r_init="constant",
    r_value=0.0
)

# Create initial state
state = model.get_state(grid)

dt = 1#0.125 * res**2 / model.dr
T = 1e5
# Set up visualization tracker
tracker = PlotTracker(interval=100*dt)

# Run the simulation
result = model.solve(state, t_range=T, dt=dt, tracker=["progress", tracker])

# Optional: display final state
state = model.get_state(grid)
state.plot()
plt.suptitle("Final State of Activator–Repressor System")
plt.show()

In [ ]:
dt